# Part 2.2
Train a model to detect MNIST
Be "creative with your model definitions

In [2]:
import torch 
import torch.nn as nn

In [ ]:
# = Definition

# Model definition

# Loss function definition

# Optimizer definition

In [ ]:
# Load data

losses_over_time = []
# Training loop

# Add the loss 

# Plot the loss_over_time graph


In [ ]:
# Save the model to .pth

# Part 2.3
Do write a lot of code snippets to make sure that things are correct.

Leave them here so that we could see that you have tried



In [1]:
# Here you will try out how to access directories

# Get the current directory
# Directory manipulation
import os

test_directory = "MNIST - JPG - testing"
training_directory = "MNIST - JPG - training"

print(os.getcwd())

full_test_directory = os.path.join(os.getcwd(), test_directory)
print(full_test_directory)


# Get all the names in the directory
# Explore how to use this glob library to get all image name
import glob



d:\NTU\MLDA-2024\Homework
d:\NTU\MLDA-2024\Homework\MNIST - JPG - testing


## Dataset and DataLoader
Follow this link https://pytorch.org/tutorials/beginner/basics/data_tutorial.html 
- Define the Dataset Class with 3 functions to get item
- Use the Dataloader in the training loop
- Iterate through the dataloader in the training loop